# Simulation
In this notebook, I simulate the interaction with the LUCE smart contract.

### Setup

In [1]:
# Import libraries
import json
import web3 
import threading
import time
from datetime import datetime
import pandas as pd
import random
import logging

from web3 import Web3
from solcx import compile_source
from web3.contract import ConciseContract

In [15]:
# Use Ganache for web3 instance
#w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
w3 = Web3(Web3.HTTPProvider("HTTP://192.168.72.1:7545"))

### Identity Management

In [16]:
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

In [17]:
# Assign of accounts
accounts = w3.eth.accounts
acNr = len(accounts)
reqAcc = accounts[2:acNr]

# Create two dictionaries for the accounts - one in each direction: (address) <> (name) and (name) <> (address)
knownAccounts={}
knownAccounts2={}

# First two accounts are the provider and authority
knownAccounts["dataProvider"] = accounts[0]
knownAccounts["authority"] = accounts[1]

x=0
for x in range(0,len(reqAcc)):
        knownAccounts["dataRequester{0}".format(x)]=reqAcc[x]

# The above, but reversed.
knownAccounts2[accounts[0]] = "dataProvider"
knownAccounts2[accounts[1]] ="authority"

x=0
for x in range(0,len(reqAcc)):
        knownAccounts2[reqAcc[x]]="dataRequester{0}".format(x)

## Functions

### Contract Deployment

In [5]:
# To be able to quickly create a new contract, I define the newContract function
def newContract():
    
    # Read in LUCE contract code
    with open('./data/luce_newFunctions.sol', 'r') as file:
        contract_source_code = file.read()
    
    # Compile & Store Compiled source code
    compiled_sol = compile_source(contract_source_code)

    # Extract full interface as dict from compiled contract
    contract_interface = compiled_sol['<stdin>:Dataset']

    # Extract abi and bytecode
    abi = contract_interface['abi']
    bytecode = contract_interface['bin']

    # Create contract blueprint
    Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

    # Submit the transaction that deploys the contract
    tx_hash = Luce.constructor().transact()

    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    global contract_block
    contract_block = None
    contract_block = w3.eth.blockNumber
    print("The contract is deployed with block number", contract_block,".")
    
    global luce1
    luce1 = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],) # Create python instance of contract

    # Obtain address of freshly deployed contract
    global contract_address
    contract_address = None
    contract_address = tx_receipt.contractAddress
    print("The new contract has the address", contract_address,".")
    
    ;



### Log Creation
Appends a row of interaction info for a given transaction hash. This function is used in the contstruction of an event log.

#### Append log

In [6]:
# In development, it was useful to quickly overwrite the log with an empty dataframe
def clearLog():
    global log
    log = pd.DataFrame(columns=['UserID', 'TaskID', 'prevTaskID', 'Timestamp', 'prevTimestamp','CaseID']) #create empty log
    ;

In [7]:
# Defining the event logger
log = pd.DataFrame(columns=['UserID', 'TaskID', 'prevTaskID', 'Timestamp', 'prevTimestamp','CaseID']) #create empty log
activeCases=[]

def logger(tx_hash,log):
    
    Tx = str(luce1.web3.eth.getTransaction(tx_hash)) #store tx_info
    
    index = Tx.find("input") # Find the substring "input" in the Tx info
    taskID = Tx[index+11:index+19] # Extract the taskID substring

    index = str(Tx).find("from") # Find the substring "from" in the Tx info
    userID = str(Tx)[index+8:index+50] # Extract the taskID substring 

    index = Tx.find("blockHash") # Find the block hash in the Tx info
    blockHash = Tx[index+22:index+88] # Extract the blockhash substring
    Block = luce1.web3.eth.getBlock(blockHash) # Get the block info by blockhash
    
    index = str(Block).find("timestamp") # Find the timestamp in block info
    ts = str(Block)[index+12:index+22] # Extract the (block) timestamp from block info
    timestamp = datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d %H:%M:%S') # Format unix to Date Time

    # prevTaskID = take the log, move backwards, get first entry of same userID    
    x = len(log)-1
    prev_taskID = None
    prev_timestamp = None
    while x >= 0 and prev_taskID is None: 
        if log.loc[x, "UserID"] == knownAccounts2[userID] and log.loc[x, "Timestamp"] < timestamp:
            prev_taskID = log.loc[x, "TaskID"]
            prev_timestamp = log.loc[x, "Timestamp"]
        else:
            ;
        x = x-1
    
    # CaseID - Used to enrich the event log with information on the relationships of the users
    global activeCases # Empty activeCases list is created in createLog() -> Compliant users
    global lockedUsers # Empty lockedUsers list in created in createLog() -> Non-compliant users
    
    y=0
    caseID = None
    
    if userID == knownAccounts["dataProvider"]: #In the general case, the providers actions address all users
        caseID = list(range(len(reqAcc)))
        
        if knownTasks[taskID] == "Request data update" or knownTasks[taskID] == "Inspect updates": # In these functions, only compliant users are addressed
            caseID = [case for case in list(range(len(reqAcc))) if case not in lockedUsers] 
        
        if knownTasks[taskID] == "Lock non-compliant users":
            caseID = [case for case in list(range(len(reqAcc))) if case not in activeCases and case not in lockedUsers] # The function addresses anyone who has not complied with the last order
            lockedUsers = lockedUsers + caseID
        

    else:
        if userID == knownAccounts["authority"]: # Authority may address all non-compliant users
            caseID = [ case for case in list(range(len(reqAcc))) if case not in activeCases]
        
        else:
            
            while y < len(reqAcc) and caseID is None: # Requesters only have one caseID
                
                if userID == reqAcc[y]:
                    caseID = y
                    
                    if knownTasks[taskID] == "Update license compliance" or knownTasks[taskID] == "Confirm update":
                        activeCases.append(caseID) # If requesters comply, they remain active     
                        
                y+=1
            ;
        ;
    
    if knownTasks[taskID] == 'Lock non-compliant users': # Once the "deadline" to comply has passed, non-compliant are locked
        activeCases = [] #The active cases reset and users have to add themselves to the list by complying again
        
    #Transaction ID
    global transactionID #get global index
    transactionID = transactionID + 1 #increment by 1

    
    #Appening variables to the log
    if type(caseID) == int: 
            log = log.append({'UserID':userID, 'TaskID':taskID, 'prevTaskID':prev_taskID, 'Timestamp':timestamp, 'prevTimestamp':prev_timestamp, 'CaseID':caseID, 'TransactionID':transactionID}, ignore_index=True)
    else:
        for z in caseID: #If the caseID is a list, it duplicates the record for every case
            log = log.append({'UserID':userID, 'TaskID':taskID, 'prevTaskID':prev_taskID, 'Timestamp':timestamp, 'prevTimestamp':prev_timestamp, 'CaseID':z, 'TransactionID':transactionID}, ignore_index=True)

    log["UserID"].replace(knownAccounts2, inplace=True) #This is likely a duplicate and not necessary, but I dont want to break anything so I will leave it.
    return log


##### TXhashes of address
Gets a list of transaction hashes associated with (to or from) a given address. Used in the construction of an event log.

In [8]:
def get_all_transactions(_address): #INPUT any address as string, get all transaction hashes. May take long!
    contract_txs=[] # create empty list for hashes
    
    i= contract_block-1
    strAddresss = str(_address) # attempted workaround the string requirement but ineffective
    
    while i < w3.eth.blockNumber+1: #loop through blocks since contract deployment
        
        block = w3.eth.getBlock(i,'true') # Get block info
        
        trans = block.transactions # Focus down on transaction part
        
        strtrans = str(trans) # Convert tx info to string
        
        spltrans = strtrans.split("}") # split txinfo into individual transactions (useful when more traffic)
        
        l = 0
        
        while l < len(spltrans): # loop through transactions (now only one, but loop to "future proof")
            
            if strAddresss in spltrans[l]: # if address appears in transaction (sender or receiver!), get hash
                start = spltrans[l].find("{")+19
                stop = spltrans[l].find("nonce")-5
                contract_txs = contract_txs + [spltrans[l][start:stop]]
    
            else:
                ; # if not, move on
            l=l+1
        i=i+1
    

    return contract_txs; # output list of all TX hashes

##### Post-mortem Event Log
Create an event log from a list of TX hashes. Used in the construction of an event log.

In [9]:
def post_mortem_log(txhash_list, log): #Post mortem, as the log is created AFTER the transactions have been made.
    for i in range(len(txhash_list)): # Cycle through hash list and log every entry
        log = logger(txhash_list[i], log)
    return log;

#### [Complete] Create Event Log
Creates a complete event log of the interaction with a given address. Best used with contract addresses.


In [10]:
#Scans all blocks for transactions involving a given address

def createLog(_address):
    all_tx = get_all_transactions(_address) # Get list of all transaction hashes associated with address from all blocks (!)
    
    log = pd.DataFrame(columns=['UserID', 'TaskID', 'prevTaskID', 'Timestamp', 'prevTimestamp','CaseID','TransactionID']) #create empty log    

    global activeCases
    activeCases=[] #Define activeCases used in logger()
    
    global lockedUsers
    lockedUsers = [] # Define lockedUsers used in logger()
    
    global transactionID
    transactionID = -1 #Start the index at zero (first tx will be 0) #This makes clear that a transaction only happened once
    
    log = post_mortem_log(all_tx, log) # Create a new log from transaction info

    #Enrich the log with external information - necessary if using caseID but not strictly needed if not using caseID.
    log["UserID"].replace(knownAccounts2, inplace=True) 
    log["TaskID"].replace(knownTasks, inplace=True)
    log["prevTaskID"].replace(knownTasks, inplace=True)

    return log;



## Simulation


### Contract Functions

In [11]:
knownTasks = {} # Define the dictionary for task names. 
# In live setting: Can be derived from Solidity code (function names) and associated transactions



def _publishData(): 
       
    tx_hash = luce1.functions.publishData("example description", "my link", 2).transact({'from': accounts[0]})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    global knownTasks
    knownTasks = {}
    Tx = str(luce1.web3.eth.getTransaction(tx_hash)) #store tx_info
    index = Tx.find("input") # Find the substring "input" in the Tx info
    taskID = Tx[index+11:index+19] # Extract the taskID substring
    knownTasks[taskID] = "Publish Dataset"
    global knownTasks2
    knownTasks2 = {v: k for k, v in knownTasks.items()};
    
       
def _submitPurpose(reqnr):
    tx_hash = luce1.functions.submitPurpose().transact({'from': reqAcc[reqnr]})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    global knownTasks
    Tx = str(luce1.web3.eth.getTransaction(tx_hash)) #store tx_info
    index = Tx.find("input") # Find the substring "input" in the Tx info
    taskID = Tx[index+11:index+19] # Extract the taskID substring
    knownTasks[taskID] = "Submit Purpose"
    global knownTasks2
    knownTasks2 = {v: k for k, v in knownTasks.items()};
    
    
def _agreeLicense(reqnr):
    tx_hash = luce1.functions.agreeLicense(2,2).transact({'from': reqAcc[reqnr]})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

    
    global knownTasks
    Tx = str(luce1.web3.eth.getTransaction(tx_hash)) #store tx_info
    index = Tx.find("input") # Find the substring "input" in the Tx info
    taskID = Tx[index+11:index+19] # Extract the taskID substring
    knownTasks[taskID] = "Agree to License"
    global knownTasks2
    knownTasks2 = {v: k for k, v in knownTasks.items()};
    
    
def _getLink(reqnr):
    tx_hash = luce1.functions.getLink(1).transact({'from': reqAcc[reqnr]})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    global knownTasks
    Tx = str(luce1.web3.eth.getTransaction(tx_hash)) #store tx_info
    index = Tx.find("input") # Find the substring "input" in the Tx info
    taskID = Tx[index+11:index+19] # Extract the taskID substring
    knownTasks[taskID] = "Get Link"
    global knownTasks2
    knownTasks2 = {v: k for k, v in knownTasks.items()};
    
    
def _notifyError(reqnr):
    tx_hash = luce1.functions.notifyError().transact({'from': reqAcc[reqnr]})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    global knownTasks
    Tx = str(luce1.web3.eth.getTransaction(tx_hash)) #store tx_info
    index = Tx.find("input") # Find the substring "input" in the Tx info
    taskID = Tx[index+11:index+19] # Extract the taskID substring
    knownTasks[taskID] = "Notify of error"
    global knownTasks2
    knownTasks2 = {v: k for k, v in knownTasks.items()};
    

def _updateCompliance(reqnr):
    tx_hash = luce1.functions.updateCompliance(2).transact({'from': reqAcc[reqnr]})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash) 
    
    global knownTasks
    Tx = str(luce1.web3.eth.getTransaction(tx_hash)) #store tx_info
    index = Tx.find("input") # Find the substring "input" in the Tx info
    taskID = Tx[index+11:index+19] # Extract the taskID substring
    knownTasks[taskID] = "Update license compliance"
    global knownTasks2
    knownTasks2 = {v: k for k, v in knownTasks.items()};
    
    
    
def _updateData():
    tx_hash = luce1.functions.updateData("New Description", "New Link").transact({'from': accounts[0]})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    global knownTasks
    Tx = str(luce1.web3.eth.getTransaction(tx_hash)) #store tx_info
    index = Tx.find("input") # Find the substring "input" in the Tx info
    taskID = Tx[index+11:index+19] # Extract the taskID substring
    knownTasks[taskID] = "Request data update"
    global knownTasks2
    knownTasks2 = {v: k for k, v in knownTasks.items()};
    
def _confirmUpdate(reqnr):
    tx_hash = luce1.functions.confirmUpdate().transact({'from': reqAcc[reqnr]})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    global knownTasks
    Tx = str(luce1.web3.eth.getTransaction(tx_hash)) #store tx_info
    index = Tx.find("input") # Find the substring "input" in the Tx info
    taskID = Tx[index+11:index+19] # Extract the taskID substring
    knownTasks[taskID] = "Confirm update"
    global knownTasks2
    knownTasks2 = {v: k for k, v in knownTasks.items()};
    
def _inspectUpdate(provORauth):
    tx_hash = luce1.functions.inspectUpdate().transact({'from': accounts[provORauth]})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    global knownTasks
    Tx = str(luce1.web3.eth.getTransaction(tx_hash)) #store tx_info
    index = Tx.find("input") # Find the substring "input" in the Tx info
    taskID = Tx[index+11:index+19] # Extract the taskID substring
    knownTasks[taskID] = "Inspect updates"
    global knownTasks2
    knownTasks2 = {v: k for k, v in knownTasks.items()};
    
def _lockNoncompliant(provORauth):
    tx_hash = luce1.functions.lockNoncompliant().transact({'from': accounts[provORauth]})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    global knownTasks
    Tx = str(luce1.web3.eth.getTransaction(tx_hash)) #store tx_info
    index = Tx.find("input") # Find the substring "input" in the Tx info
    taskID = Tx[index+11:index+19] # Extract the taskID substring
    knownTasks[taskID] = "Lock non-compliant users"
    global knownTasks2
    knownTasks2 = {v: k for k, v in knownTasks.items()};

### Stepwise Base Functions

In [12]:
def step1_prov():
    # Publishing dataset (or correct error)
    time.sleep(random.randint(1,10))
    _publishData()
    ;
    
def step2_req(reqAcc):
    # Accessing dataset
    
    time.sleep(random.randint(15,30))
    _agreeLicense(reqAcc)
    
    time.sleep(random.randint(5,20))
    _submitPurpose(reqAcc)
    
    time.sleep(random.randint(5,20))
    _getLink(reqAcc)
    
    ;
    
def step2stop_req(reqAcc):
    # Never get the link, lost interest, end flow

    time.sleep(random.randint(15,30))
    _agreeLicense(reqAcc)
    
    time.sleep(random.randint(5,20))
    _submitPurpose(reqAcc)
    
    time.sleep(random.randint(5,20))
    
    ;
    
def step3_req(reqnr):
    # Notify provider of error with dataset
    
    time.sleep(random.randint(15,30))
    _notifyError(reqnr)
    
    ;
    
def step4_req(reqnr): 
    # Update compliance behavior periodically --> Repeated a few times

    time.sleep(random.randint(20,50)) # The response time differs between user
    
    _updateCompliance(reqnr)
    
    ;
    
def step4B_prov(provORauth):
    # Lock out non-compliant users
    
    time.sleep(random.randint(5,20))
    _lockNoncompliant(provORauth)
    
    ;
    

def step5_prov():
    # Request update from all requesters
    
    time.sleep(random.randint(15,30))
    _updateData()
    
    ;
    
def step6_req(reqnr):
    # Confirm update of dataset to provider
    
    time.sleep(random.randint(5,50)) #vastly different response times
    _confirmUpdate(reqnr)
    
    ;
    
def step7_provauth(provORauth):
    # Inspect the updates of the requesters (who has, who hasnt updated)
    
    time.sleep(random.randint(15,30))
    _inspectUpdate(provORauth)
    
    ;
    
    
def step8_provauth(provORauth):
    # Lock out non-compliant users
    
    time.sleep(random.randint(5,15))
    _lockNoncompliant(provORauth)
    
    ; 
    



### Control Flow Simulation
Threaded execution of the control flow involving requester accounts.

### Variant 1
Includes slight variation to the base model.

In [ ]:
newContract() # Deploys a new contract

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

threads = list() 

logging.info("Provider starts sharing dataset.")
step1_prov()

# users = list(range(len(reqAcc)))
# idiots2 = random.sample(user, 2)
# reduced_2 = [case for case in user if case not in idiots2]
# forgetful3 = random.sample(reduced_2, 3)
# reduced_5 =  [case for case in user if case not in forgetful3]
# reduced_10 = [case for case in reduced_5 if case not in random.sample(reduced_5, 10)]
# reduced_20 = [case for case in reduced_10 if case not in random.sample(reduced_10, 20)]
# reduced_23 = [case for case in reduced_20 if case not in random.sample(reduced_20, 3)]

users = list(range(len(reqAcc))) # Grouping the requesters for threading
idiots2 = [97, 96]
reduced_2 = list(range(96))
forgetful3 = [95,94,93]
reduced_5 =  list(range(93))
reduced_10 = list(range(88))
reduced_20 = list(range(78))
reduced_23 = list(range(75))


for req in reduced_2:
    logging.info("Main    : Starting step2 for requester %d.", req)
    x = threading.Thread(target = step2_req, args=(req,))
    threads.append(x)
    x.start()
    
for req in idiots2:
    logging.info("Main    : Starting step2_reduced for forgetters %d.", req)
    t = threading.Thread(target = step2stop_req, args=(req,))
    threads.append(t)
    t.start()  

for i, thread in enumerate(threads):
        thread.join()
        logging.info("Main    : Requester %d done.", i)
        
for req in forgetful3:
    t=threading.Thread(target  = step3_req, args = (req,))
    threads.append(t)
    t.start()
             
for i, thread in enumerate(threads):
        thread.join()
        logging.info("Main    : Requester %d done.", i)    

time.sleep(30)

for req in reduced_10:
    x = threading.Thread(target = step4_req, args=(req,))
    threads.append(x)
    logging.info("Main    : Reporting compliance for requester %d.", req)
    x.start()

for i, thread in enumerate(threads):
        thread.join()
        logging.info("Main    : Requester %d done.", i)    
             
logging.info("Provider locks noncompliant.")
step4B_prov(0)

time.sleep(30) 

for req in reduced_20:
    x = threading.Thread(target = step4_req, args=(req,))
    threads.append(x)
    logging.info("Main    : Reporting compliance for requester %d.", req)
    x.start()

for i, thread in enumerate(threads):
        thread.join()
        
logging.info("Provider locks noncompliant.")
step4B_prov(0)
                  
time.sleep(30)
                  
logging.info("Provider requests update.")           
step5_prov()
             
for req in reduced_23:
    x = threading.Thread(target = step6_req, args=(req,))
    threads.append(x)
    logging.info("Main    : Confirming update for requester %d.", req)
    x.start()

for i, thread in enumerate(threads):
        thread.join()
                  
time.sleep(5)
                  
for pOa in range(2):
    x = threading.Thread(target = step7_provauth, args=(pOa,))
    threads.append(x)
    logging.info("Main    : Inspecting update.", pOa)
    x.start()

for i, thread in enumerate(threads):
        thread.join()

step8_provauth(0)

time.sleep(20)
             
for req in reduced_23:
    x = threading.Thread(target = step4_req, args=(req,))
    threads.append(x)
    logging.info("Main    : Update compliance for requester %d.", req)
    x.start()

for i, thread in enumerate(threads):
        thread.join()

step4B_prov(0)



In [564]:
simulation = createLog(contract_address)

simulation


,UserID,TaskID,prevTaskID,Timestamp,prevTimestamp,CaseID,TransactionID
0,dataProvider,Publish Dataset,None,2019-08-10 20:54:07,None,0,0
1,dataProvider,Publish Dataset,None,2019-08-10 20:54:07,None,1,0
2,dataProvider,Publish Dataset,None,2019-08-10 20:54:07,None,2,0
3,dataProvider,Publish Dataset,None,2019-08-10 20:54:07,None,3,0
4,dataProvider,Publish Dataset,None,2019-08-10 20:54:07,None,4,0
5,dataProvider,Publish Dataset,None,2019-08-10 20:54:07,None,5,0
6,dataProvider,Publish Dataset,None,2019-08-10 20:54:07,None,6,0
7,dataProvider,Publish Dataset,None,2019-08-10 20:54:07,None,7,0
8,dataProvider,Publish Dataset,None,2019-08-10 20:54:07,None,8,0
9,dataProvider,Publish Dataset,None,2019-08-10 20:54:07,None,9,0


In [565]:
simulation.to_csv("simulation.csv")